In [1]:
from datetime import datetime
from datetime import timedelta
import glob
import pandas as pd
import plotly.graph_objects as go
import os  
  
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

Let's first plot Luna price and then overlay with MAs of certain time intervals

In [2]:
# loop over the list of csv files
ldf = [] #used to concat all csv

for f in sorted(csv_files):
    cdf = pd.read_csv(f) #current data frame(per csv)
    # print(df.head(1))
    # print('--------------')
    cdf_OHLCV = cdf.iloc[:,0:6]
    
    for i in range(cdf_OHLCV.shape[0]):
        ts = cdf_OHLCV.iloc[i,0] #timestamp
        ts_edit = datetime.strptime(ts, '%Y-%m-%dT%H:%M:%SZ').date()
        cdf_OHLCV.iloc[i,0] = ts_edit.strftime("%Y/%m/%d")

    ldf.append(cdf_OHLCV)

df= pd.concat(ldf)
df = df.sort_values(by='timestamp').reset_index(drop=True)

In [6]:
times = list(df.iloc[:,0])
open_data = list(df.iloc[:,1])
high_data = list(df.iloc[:,2])
low_data = list(df.iloc[:,3])
close_data = list(df.iloc[:,4])

coin = 'LUNA'
fig = go.Figure(data=[go.Candlestick(x=times,
                                    open=open_data, 
                                    high=high_data,
                                    low=low_data, 
                                    close=close_data)])

fig.update_layout(
                title={
                    'text' : coin + '/USD 1d from ' + df.iloc[0,0] + ' to ' + df.iloc[-1,0],
                    'y':0.92,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title="Time",
                yaxis_title= coin + '/USD',
                width = 800,
                height = 600)

fig.show()

In [8]:
#Plot and n_moving average plot
times = list(df.iloc[:,0])
open_data = list(df.iloc[:,1])
high_data = list(df.iloc[:,2])
low_data = list(df.iloc[:,3])
close_data = list(df.iloc[:,4])
coin = 'LUNA'
fig = go.Figure(data=[go.Candlestick(x=times,
                                    open=open_data, 
                                    high=high_data,
                                    low=low_data, 
                                    close=close_data)])

n = 30
n_day_MA = []
for i in range(df.shape[0]):
    if i + n < df.shape[0]:
        summer = 0
        ma = []
        for j in range(n):
            summer += df.iloc[i+j,1]
            ma.append(summer/(j+1))
        n_day_MA.append(ma)
for i in range(len(n_day_MA)):
    fig.add_trace(go.Scatter(
        x=times[i:i+n+1],
        y=n_day_MA[i],
        name="Name of Trace 2"
    ))

fig.update_layout(
                title={
                    'text' : coin + '/USD 1d from ' + df.iloc[0,0] + ' to ' + df.iloc[-1,0],
                    'y':0.92,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title="Time",
                yaxis_title= coin + '/USD',
                width = 800,
                height = 600)

fig.show()